In [1]:
from drive.google_main import Google
from drive.error import Error

class file(Google):
    def __init__(self,file_name) -> None:
        super().__init__()
        # puxando nome do arquivo que será substituido
        self.__file = file_name
        self.service = self.generate_service()

    def generate_service(self):
        #Keys para capturar pastas drive
        SCOPES = ['https://www.googleapis.com/auth/drive.file']
        CLIENT_SECRET_FILE = self.chromedriver
        API_NAME = 'drive'
        API_VERSION = 'V3'
        return self.Create_Service(CLIENT_SECRET_FILE,API_NAME,API_VERSION,SCOPES)
     
    def move_file(self,folder_id,id=143509):
        '''
            more then one file -> for this method the code expect to receive a list or a dic with the itens,\n
                if you want to send all the itens to the same folder, you can use the function like:\n
                >>> objt = file([file1,file2,folder_dest])
                >>> move_file(folder_id)
                \n
                but if you want to send each item for a diferent folder, you can use the function like:\n
                >>> objt = file({file1:folderkey,file2:folderkey})
                >>> move_file(folder_id)
                \n
                the function expect to receive all folder keys to each item
        '''
        # proceso para lista de itens
        if type(self.__file) == list:
            
            # chechando se a lista esta correta 
            if len(self.__file)<=1:
                Error(id,"len<1",self.__file)
                return
            key_dest = 
            for file in self.__file[0:-1]:
                query = f"parents = '{folder_id}' and name = '{file}'"
                response = self.service.files().list(q=query).execute()
                print(file)

        elif type(self.__file) == str:
            query = f"parents = '{folder_id}' and name = '{self.__file}'"
            response = self.service.files().list(q=query).execute()
            print(response)
        elif type(self.__file) == dict:
            for name_item in self.__file:
                key_dest = self.__file[name_item]
                print(key_dest)
# teste=file({'1197769':"tes",'1197768':'pá'})

In [4]:
teste=file(['sfawf'])
teste.move_file('15oT4oGeFL1_AFoj53FfU781vnULb21y9')

ERROR
the code expected to receive a list with one or more items and the key dest, but received a list with just one element that can not be specified
the element is: ['sfawf']


TypeError: file.move_file() missing 1 required positional argument: 'folder_id'